#  Data Wrangling Join, Combine, and Reshape

In [26]:
import numpy as np
import pandas as pd

In [27]:
np.random.seed(0)

## Hierarchical Indexing

Enables you to have multiple (two or more) index levels on an axis.
Detailed Tutorials:

- [http://www.sefidian.com/2021/06/24/data-selection-in-a-multiindex-dataframe-in-pandas/](http://www.sefidian.com/2021/06/24/data-selection-in-a-multiindex-dataframe-in-pandas/)
- [http://www.sefidian.com/2021/06/24/a-complete-guide-on-pandas-hierarchical-indexing-multiindex/](http://www.sefidian.com/2021/06/24/a-complete-guide-on-pandas-hierarchical-indexing-multiindex/)

In [28]:
data = pd.Series(
    np.random.randn(9),
    index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"], [1, 2, 3, 1, 3, 1, 2, 2, 3]],
)

In [29]:
data

a  1    1.764052
   2    0.400157
   3    0.978738
b  1    2.240893
   3    1.867558
c  1   -0.977278
   2    0.950088
d  2   -0.151357
   3   -0.103219
dtype: float64

In [30]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [31]:
data["b"]

1    2.240893
3    1.867558
dtype: float64

In [32]:
data["b":"c"]

b  1    2.240893
   3    1.867558
c  1   -0.977278
   2    0.950088
dtype: float64

In [33]:
data.loc[["b", "d"]]

b  1    2.240893
   3    1.867558
d  2   -0.151357
   3   -0.103219
dtype: float64

In [34]:
data[["b", "d"]]

b  1    2.240893
   3    1.867558
d  2   -0.151357
   3   -0.103219
dtype: float64

In [35]:
data

a  1    1.764052
   2    0.400157
   3    0.978738
b  1    2.240893
   3    1.867558
c  1   -0.977278
   2    0.950088
d  2   -0.151357
   3   -0.103219
dtype: float64

In [36]:
data.loc[["b", "d"], :]

b  1    2.240893
   3    1.867558
d  2   -0.151357
   3   -0.103219
dtype: float64

In [37]:
data.loc[:, 1, :]

a    1.764052
b    2.240893
c   -0.977278
dtype: float64

In [38]:
data.loc[:, 2]

a    0.400157
c    0.950088
d   -0.151357
dtype: float64

You could rearrange the data into a DataFrame using its `unstack` method:

In [39]:
data.unstack()

,1,2,3
a,1.764052,0.400157,0.978738
b,2.240893,NaN,1.867558
c,-0.977278,0.950088,NaN
d,NaN,-0.151357,-0.103219


The inverse operation is `stack`:

In [40]:
data.unstack().stack()

a  1    1.764052
   2    0.400157
   3    0.978738
b  1    2.240893
   3    1.867558
c  1   -0.977278
   2    0.950088
d  2   -0.151357
   3   -0.103219
dtype: float64

With a DataFrame, either axis can have a hierarchical index:

In [41]:
frame = pd.DataFrame(
    np.arange(12).reshape((4, 3)),
    index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
    columns=[["Ohio", "Ohio", "Colorado"], ["Green", "Red", "Green"]],
)

In [42]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [43]:
frame.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [44]:
frame.columns

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           )

In [45]:
frame.loc[["a"]]

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5

In [46]:
frame.loc[["a"]].iloc[1, [0, 2]]

Ohio      Green    3
Colorado  Green    5
Name: (a, 2), dtype: int64

In [47]:
frame.loc[["a"], (slice(None), "Green")]

Ohio Colorado
    Green    Green
a 1     0        2
  2     3        5

The hierarchical levels can have names (as strings or any Python objects). If so, these will show up in the console output:

In [48]:
frame.index.names = ["key1", "key2"]

In [49]:
frame.columns.names = ["state", "color"]

In [50]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

With partial column indexing you can similarly select groups of columns:

In [51]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

A MultiIndex can be created by itself and then reused; the columns in the preceding
DataFrame with level names could be created like this:

In [52]:
pd.MultiIndex.from_arrays(
    [["Ohio", "Ohio", "Colorado"], ["Green", "Red", "Green"]], names=["state", "color"]
)

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

The `swaplevel` takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise
unaltered):

In [53]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` sorts the data using only the values in a single level.

When swapping levels, it's not uncommon to also use sort_index so that the result is lexicographically sorted by the indicated level:

In [54]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [55]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [56]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [57]:
frame.swaplevel(0, 1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [58]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and Series have a `level` option in which you can specify the level you want to aggregate by on a particular axis. Consider the above DataFrame; we can aggregate by level on either the rows or columns like so:

In [59]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [60]:
frame.sum(level="key2")

/home/masoud/anaconda3/envs/coding_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """Entry point for launching an IPython kernel.


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [61]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [62]:
frame.sum(level="color", axis=1)

/home/masoud/anaconda3/envs/coding_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """Entry point for launching an IPython kernel.


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame’s columns

you may wish to move the row index into the DataFrame’s columns. Here’s an example DataFrame:

In [63]:
frame = pd.DataFrame(
    {
        "a": range(7),
        "b": range(7, 0, -1),
        "c": ["one", "one", "one", "two", "two", "two", "two"],
        "d": [0, 1, 2, 0, 1, 2, 3],
    }
)

In [64]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame's `set_index` function will create a new DataFrame using one or more of its columns as the index:

In [65]:
frame3 = frame.set_index(["c", "d"])

In [66]:
frame3

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [67]:
frame2 = frame.set_index(["c", "d"], drop=False)

In [68]:
frame2

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index`, on the other hand, does the opposite of set_index; the hierarchical index levels are moved into the columns:

In [69]:
frame3.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Datasets

* `pandas.merge` connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.
* `pandas.concat` concatenates or “stacks” together objects along an axis.
* The `combine_first` instance method enables splicing together overlapping data to fill in missing values in one object with values from another.

### Database-Style DataFrame Joins

The `merge` function in pandas is the main entry point for using these algorithms on your data.

In [70]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], "data1": range(7)})
df2 = pd.DataFrame({"key": ["a", "b", "d"], "data2": range(3)})

In [71]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [72]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [73]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


**If that information is not specified, merge uses the overlapping column names as the keys.** 

In [74]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, you can specify them separately:

In [75]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"], "data1": range(7)})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"], "data2": range(3)})

In [76]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [77]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [78]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


By default merge does an 'inner' join; the keys in the result are the intersection, or the common set found in both tables. Other possible options are 'left','right', and 'outer'. The outer join takes the union of the keys, combining the effect of applying both left and right joins:

Different join types with how argument:

'inner' Use only the key combinations observed in both tables

'left' Use all key combinations found in the left table

'right' Use all key combinations found in the right table

'output' Use all key combinations observed in both tables together

In [79]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [80]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [81]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [82]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], "data1": range(6)})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"], "data2": range(5)})

In [83]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [84]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [85]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [86]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


To merge with multiple keys, pass a list of column names:

In [87]:
left = pd.DataFrame(
    {"key1": ["foo", "foo", "bar"], "key2": ["one", "two", "one"], "lval": [1, 2, 3]}
)
right = pd.DataFrame(
    {
        "key1": ["foo", "foo", "bar", "bar"],
        "key2": ["one", "one", "one", "two"],
        "rval": [4, 5, 6, 7],
    }
)

In [88]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [89]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [90]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


merge has a `suffixes` option for specifying strings to append to overlapping names in the left and right DataFrame objects:

In [91]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [92]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [93]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [94]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


![alt text](images/merge.png " merge function arguments")

### Merging on Index

In some cases, the merge key(s) in a DataFrame will be found in its index. In this case, you can pass `left_index=True` or `right_index=True` (or both) to indicate that the index should be used as the merge key:

In [95]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"], "value": range(6)})

In [96]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [97]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [98]:
right1

,group_val
a,3.5
b,7.0


In [99]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [100]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


With hierarchically indexed data, things are more complicated, as joining on index is implicitly a multiple-key merge:

In [101]:
lefth = pd.DataFrame(
    {
        "key1": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
        "key2": [2000, 2001, 2002, 2001, 2002],
        "data": np.arange(5.0),
    }
)
righth = pd.DataFrame(
    np.arange(12).reshape((6, 2)),
    index=[
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002],
    ],
    columns=["event1", "event2"],
)

In [102]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [103]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In this case, you have to indicate multiple columns to merge on as a list (note the handling of duplicate index values with how='outer'):

In [104]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [105]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


Using the indexes of both sides of the merge is also possible:

In [106]:
left2 = pd.DataFrame(
    [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]],
    index=["a", "c", "e"],
    columns=["Ohio", "Nevada"],
)
right2 = pd.DataFrame(
    [[7.0, 8.0], [9.0, 10.0], [11.0, 12.0], [13, 14]],
    index=["b", "c", "d", "e"],
    columns=["Missouri", "Alabama"],
)

In [107]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [108]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [109]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame has a convenient `join` instance for merging by index. It can also be used to combine together many DataFrame objects having the same or similar indexes but **non-overlapping** columns. In the prior example, we could have written:

In [110]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame’s join method performs a _left join_ on the join keys, exactly preserving the left frame’s row index. It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame:

In [111]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [112]:
right1

,group_val
a,3.5
b,7.0


In [113]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


for simple index-on-index merges, you can pass a list of DataFrames to join as an alternative to using the more general `concat` function described in the next section:

In [114]:
another = pd.DataFrame(
    [[7.0, 8.0], [9.0, 10.0], [11.0, 12.0], [16.0, 17.0]],
    index=["a", "c", "e", "f"],
    columns=["New York", "Oregon"],
)

In [115]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [116]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [117]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating Along an Axis

NumPy's `concatenate` function can do this with NumPy arrays:

In [118]:
arr = np.arange(12).reshape((3, 4))

In [119]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [120]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [121]:
np.concatenate([arr, arr], axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In the context of pandas objects such as Series and DataFrame, having labeled axes enable you to further generalize array concatenation. In particular, you have a number of additional things to think about:

* If the objects are *indexed differently* on the other axes, should we combine the distinct elements in these axes or use only the shared values (the intersection)?
* Do the concatenated chunks of data need to be identifiable in the resulting object?
* Does the “concatenation axis” contain data that needs to be preserved? In many cases, the default integer labels in a DataFrame are best discarded during concatenation.

The `concat` function in pandas provides a consistent way to address each of these concerns.

In [122]:
s1 = pd.Series([0, 1], index=["a", "b"])
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"])
s3 = pd.Series([5, 6], index=["f", "g"])

In [123]:
s1

a    0
b    1
dtype: int64

In [124]:
s2

c    2
d    3
e    4
dtype: int64

In [125]:
s3

f    5
g    6
dtype: int64

In [126]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

By default `concat` works along `axis=0`, producing another Series. If you pass `axis=1`, the result will instead be a DataFrame (`axis=1` is the columns):

In [127]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In this case there is no overlap on the other axis, which as you can see is the sorted union (the 'outer' join) of the indexes. You can instead intersect them by passing `join='inner'`:

In [128]:
s4 = pd.concat([s1, s3])

In [129]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [130]:
s1

a    0
b    1
dtype: int64

In [131]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [132]:
pd.concat([s1, s4], axis=1, join="inner")

,0,1
a,0,0
b,1,1


A potential issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the `keys` argument:

In [133]:
s1

a    0
b    1
dtype: int64

In [134]:
s3

f    5
g    6
dtype: int64

In [135]:
pd.concat([s1, s1, s3])

a    0
b    1
a    0
b    1
f    5
g    6
dtype: int64

In [136]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])

In [137]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [138]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In the case of combining Series along `axis=1`, the keys become the DataFrame column headers:

In [139]:
pd.concat([s1, s2, s3], axis=1, keys=["one", "two", "three"])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


The same logic extends to DataFrame objects:

In [140]:
df1 = pd.DataFrame(
    np.arange(6).reshape(3, 2), index=["a", "b", "c"], columns=["one", "two"]
)
df2 = pd.DataFrame(
    5 + np.arange(4).reshape(2, 2), index=["a", "c"], columns=["three", "four"]
)

In [141]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [142]:
df2

,three,four
a,5,6
c,7,8


In [143]:
pd.concat([df1, df2], axis=1, keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

If you pass a dict of objects instead of a list, the dict’s keys will be used for the _keys_ option:

In [144]:
pd.concat({"level1": df1, "level2": df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

we can name the created axis levels with the `names` argument:

In [145]:
pd.concat([df1, df2], axis=1, keys=["level1", "level2"], names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

![alt text](images/concat.png "concat function arguments")

A last consideration concerns DataFrames in which the row index does not contain any relevant data:

In [146]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=["a", "b", "c", "d"])

In [147]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=["b", "d", "a"])

In [148]:
df1

,a,b,c,d
0,0.410599,0.144044,1.454274,0.761038
1,0.121675,0.443863,0.333674,1.494079
2,-0.205158,0.313068,-0.854096,-2.552990


In [149]:
df2

,b,d,a
0,0.653619,0.864436,-0.742165
1,2.269755,-1.454366,0.045759


In this case, you can pass `ignore_index=True`:

In [150]:
pd.concat([df1, df2])

,a,b,c,d
0,0.410599,0.144044,1.454274,0.761038
1,0.121675,0.443863,0.333674,1.494079
2,-0.205158,0.313068,-0.854096,-2.552990
0,-0.742165,0.653619,NaN,0.864436
1,0.045759,2.269755,NaN,-1.454366


In [151]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.410599,0.144044,1.454274,0.761038
1,0.121675,0.443863,0.333674,1.494079
2,-0.205158,0.313068,-0.854096,-2.552990
3,-0.742165,0.653619,NaN,0.864436
4,0.045759,2.269755,NaN,-1.454366


### Combining Data with Overlap

You may have two datasets whose indexes overlap in full or part. As a motivating example, consider NumPy’s where function, which performs the array-oriented equivalent of an `if-else` expression:

In [152]:
a = pd.Series(
    [np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=["f", "e", "d", "c", "b", "a"]
)
b = pd.Series(np.arange(len(a), dtype=np.float64), index=["f", "e", "d", "c", "b", "a"])

In [153]:
b[-1] = np.nan

In [154]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [155]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [156]:
# fill a with b values everywhere that a is null
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [157]:
pd.isnull(a)

f     True
e    False
d     True
c    False
b    False
a     True
dtype: bool

Series has a `combine_first` method, which performs the equivalent of this operation along with pandas's usual data alignment logic:

In [158]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [159]:
a[2:]

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [160]:
b[:-2].combine_first(
    a[2:]
)  # Combine Series values, choosing the calling Series's values first.
# Result index will be the union of the two indexes

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

## Reshaping and Pivoting

There are a number of basic operations for rearranging tabular data. These are alternatingly referred to as reshape or pivot operations.

### Reshaping with Hierarchical Indexing

Hierarchical indexing provides a consistent way to rearrange data in a DataFrame.
There are two primary actions:

- stack
    This “rotates” or pivots from the columns in the data to the rows
    
- unstack
    This pivots from the rows into the columns

In [161]:
data = pd.DataFrame(
    np.arange(6).reshape((2, 3)),
    index=pd.Index(["Ohio", "Colorado"], name="state"),
    columns=pd.Index(["one", "two", "three"], name="number"),
)

In [162]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Using the `stack` method on this data pivots the columns into the rows, producing a Series:

In [163]:
result = data.stack()

In [164]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with `unstack`:

In [165]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


By default the `innermost` level is unstacked (same with `stack`). You can unstack a different level by passing a *level number* or *name*:

In [166]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [167]:
result.unstack("state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Unstacking might introduce missing data if all of the values in the level aren’t found in each of the subgroups:

In [168]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"])
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"])

In [169]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [170]:
s2

c    4
d    5
e    6
dtype: int64

In [171]:
data2 = pd.concat([s1, s2], keys=["one", "two"])

In [172]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [173]:
data2.unstack(1)

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [174]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [175]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [176]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result:

In [177]:
df = pd.DataFrame(
    {"left": result, "right": result + 5},
    columns=pd.Index(["left", "right"], name="side"),
)

In [178]:
df.unstack("state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

When calling `stack`, we can indicate the name of the axis to stack:

In [179]:
df.unstack("state").stack("side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting “Long” to “Wide” Format

A common way to store multiple time series in databases and CSV is in so-called long or stacked format. Let's load some example data and do a small amount of time series wrangling and other data cleaning:

In [180]:
data = pd.read_csv("data/macrodata.csv")

In [181]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [182]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name="date")
columns = pd.Index(["realgdp", "infl", "unemp"], name="item")
data = data.reindex(columns=columns)
data.index = periods.to_timestamp("D", "end")
ldata = data.stack().reset_index().rename(columns={0: "value"})

In [183]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


This is the so-called long format for multiple time series, or other observational data with two or more keys (here, our keys are date and item). Each row in the table represents a single observation.

Data is frequently stored this way in relational databases like MySQL, as a fixed schema (column names and data types) allows the number of distinct values in the item column to change as data is added to the table. In the previous example, date and item would usually be the primary keys (in relational database parlance), offering both relational integrity and easier joins. In some cases, the data may be more difficult to work with in this format; you might prefer to have a DataFrame containing one column per distinct item value indexed by timestamps in the date column. DataFrame's `pivot` method performs exactly this transformation:

In [184]:
pivoted = ldata.pivot("date", "item", "value")

In [185]:
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


The first two values passed are the columns to be used respectively as the row and column index, then finally an optional value column to fill the DataFrame. Suppose you had two value columns that you wanted to reshape simultaneously:

In [186]:
ldata["value2"] = np.random.randn(len(ldata))

In [187]:
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.187184
1,1959-03-31 23:59:59.999999999,infl,0.000,1.532779
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.469359
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.154947
4,1959-06-30 23:59:59.999999999,infl,2.340,0.378163
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.887786
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-1.980796
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.347912
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.156349
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,1.230291


By omitting the last argument, you obtain a DataFrame with hierarchical columns

In [188]:
pivoted = ldata.pivot("date", "item")

In [189]:
pivoted.head()

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  1.532779 -0.187184   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.378163  0.154947   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.347912 -1.980796   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.202380  1.230291   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.048553 -0.302303   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.469359  
1959-06-30 23:59:59.999999999 -0.887786  
1959-09-30 23:59:59.999999999  0.156349  
1959-12-31 23:59:59.999999999 -0.387327  
1960-03-31 23:59:59.999999999 -1.420018

In [190]:
pivoted["value"][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


Note that pivot is equivalent to creating a hierarchical index using set_index followed by a call to `unstack`:

In [191]:
unstacked = ldata.set_index(["date", "item"]).unstack("item")

In [192]:
unstacked.head(7)

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  1.532779 -0.187184   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.378163  0.154947   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.347912 -1.980796   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.202380  1.230291   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.048553 -0.302303   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2  1.950775 -1.706270   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -1.252795 -0.438074   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.469359  
1959-06-30 23:59:59.999999999 -0.887786  
1959-09-30 23:59:59.999999999  0.156349  
1959-12-31 23:59:59.999999999 -0.387327  
1960-03-31 23:59:59.999999999 -1.420018  
1960-06-30 23:59:59.999999999 -0.509652  
1960-09-30 23:59:59.999999999  0.777490

### Pivoting "Wide" to "Long" Format

An inverse operation to pivot for DataFrames is pandas.melt. Rather than transforming one column into many in a new DataFrame, it merges multiple columns into one, producing a DataFrame that is longer than the input. Let's look at an example:

In [193]:
df = pd.DataFrame(
    {"key": ["foo", "bar", "baz"], "A": [1, 2, 3], "B": [4, 5, 6], "C": [7, 8, 9]}
)

In [194]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


The 'key' column may be a group indicator, and the other columns are data values.
When using pandas.melt, we must indicate which columns (if any) are group indicators. Let's use 'key' as the only group indicator here:

In [195]:
melted = pd.melt(df, ["key"])

In [196]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


Using `pivot`, we can reshape back to the original layout:

In [197]:
reshaped = melted.pivot("key", "variable", "value")

In [198]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


Since the result of `pivot` creates an index from the column used as the row labels, we may want to use `reset_index` to move the data back into a column:

In [199]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


You can also specify a subset of columns to use as value columns:

In [200]:
pd.melt(df, id_vars=["key"], value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


`pandas.melt` can be used without any group identifiers, too:

In [201]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [202]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
